In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb
%run ../../lib/statistic-functions.ipynb

In [ ]:
import termtables as tt
import os.path

#fast for finding closest value
from bisect import *

#pip install termtables

In [ ]:
#https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
import numpy as np
def take_closest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
mean_dict = dist_average()
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
def pd_plot_lap_save_corr_new():
    for participant in participant_files:
        ID = participant[:-4]

        if str(ID) != str(50766):
            continue

        #for now, we are only looking at 50338
        add_df_338 = add_df[ID]
        # 15
        #Sjekker der alle verdiene ikke er NaNs. 
        #Gjør det samme som din tidligere funksjon, bare med timestamp som index i betraktning :P.
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]

        lapRange_df_li = []
        
        # Think this is better. Find where lap changes value by 1 and make a list of the index.
        lapTimestamp_li = list(add_df_338.index[add_df_338['lap'].diff() == 1])
        lapDistance_li = list(add_df_338['distance'][add_df_338['lap'].diff() == 1])


        # Find +30s, -15 from lap increase, and store all rows
        timedelta_30s = pd.Timedelta("00:00:30")
        timedelta_15s = pd.Timedelta("00:00:15")

        newLapDistance_li = []
        newTimestamp_li = []
        print('\n')
        print('Before: ', lapTimestamp_li)
        print('Before: ', lapDistance_li)
        i = 0
        while True:
            if i == len(mean_dict):
                break
            if len(lapDistance_li) == 0:
                newDistance = add_df_338['distance'][bisect_left(add_df_338['distance'], mean_dict[i]) - 1]
                newTimestamp = add_df_338.index[bisect_left(add_df_338['distance'], mean_dict[i]) - 1]
                newLapDistance_li.append(newDistance)
                newTimestamp_li.append(newTimestamp)
            else:
                closest_val = take_closest(lapDistance_li, mean_dict[i])
                if abs(lapDistance_li[closest_val]-mean_dict[i]) > 0.01*mean_dict[i]:
                    newDistance = add_df_338['distance'][bisect_left(add_df_338['distance'], mean_dict[i]) - 1]
                    newTimestamp = add_df_338.index[bisect_left(add_df_338['distance'], mean_dict[i]) - 1]
                    newLapDistance_li.append(newDistance)
                    newTimestamp_li.append(newTimestamp)
                else:
                    newLapDistance_li.append(lapDistance_li[closest_val])
                    newTimestamp_li.append(lapTimestamp_li[closest_val])
                    del lapDistance_li[closest_val], lapTimestamp_li[closest_val]
            i += 1
        lapTimestamp_li = newTimestamp_li
        lapDistance_li = newLapDistance_li
        for i in range(len(lapDistance_li)):
            try:
                startRow = add_df_338.index.get_loc(lapTimestamp_li[i]-timedelta_15s)
            except:
                print(ID+'  something is very wrong - startRow')
                startRow = add_df_338.index.get_loc(lapTimestamp_li[i])
            try:
                endRow = add_df_338.index.get_loc(lapTimestamp_li[i]+timedelta_30s)
            except:
                print(ID+'  something is very wrong - endRow')
                endRow = add_df_338.index.get_loc(lapTimestamp_li[i])

            lapRange_df_li.append(add_df_338.iloc[startRow:endRow])

        # Create plot for every stop
        for i in range(len(lapRange_df_li)):
            fig, ax = plt.subplots()

            trans = ax.get_xaxis_transform()

            ax2 = ax.twinx()
            ax3 = ax.twinx()
            ax4 = ax.twinx()
            ax5 = ax.twinx()
            ax6 = ax.twinx()

            rightPosition = 1.00
            leftPosition = -0.20

            rightPosition = ax_modification(ax2,'right', rightPosition)
            rightPosition = ax_modification(ax3,'right', rightPosition)
            leftPosition = ax_modification(ax4,'left', leftPosition)
            rightPosition = ax_modification(ax5,'right', rightPosition)
            leftPosition = ax_modification(ax6,'left', leftPosition)
            
            plot_title = "lap {}: -15,+30 pd {}".format(i,ID)

            lapRange_df_li[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title=plot_title)
            lapRange_df_li[i]['speed'].plot(ax=ax2, color='blue',legend=False)
            lapRange_df_li[i]['power'].plot(ax=ax3, color='green',legend=False)
            lapRange_df_li[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
            lapRange_df_li[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
            lapRange_df_li[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)    
            ax.axvline((lapTimestamp_li[i]), color='black', linestyle='--', lw=2)
            ax.text((lapTimestamp_li[i]),.5,(lapTimestamp_li[i]),transform=trans)

            #dårlige runtime, trenge sikkert ikkje sjekka kver einaste value.
            #bør tenka på ein bedre måte
            check_ylim(ax,lapRange_df_li[i],'accumulated_power')
            check_ylim(ax2,lapRange_df_li[i],'speed')
            check_ylim(ax3,lapRange_df_li[i],'power')
            check_ylim(ax4,lapRange_df_li[i],'altitude')
            check_ylim(ax5,lapRange_df_li[i],'distance')

            y_lim_zero(ax)
            y_lim_zero(ax2)
            y_lim_zero(ax3)
            y_lim_zero(ax4)
            y_lim_zero(ax5)
            y_lim_zero(ax6)

            ax_style(ax,'Accumulated Power','red')
            ax_style(ax2,'Speed','blue')
            ax_style(ax3,'Power','green')
            ax_style(ax4,'Altitude','orange')
            ax_style(ax5,'Distance','lightcoral')
            ax_style(ax6,'Heartrate','purple')
            
            #bbox_inches makes it possible to see all y axis in image.
            #all the y axis didnt show pdf when incrementing rightposition and leftposition.
            #however think we need to resize it to fit in the pdf.
            fig.savefig("lap{}pd{}.png".format(i,ID), format="png", bbox_inches='tight')

            #trying to hide plots from notebook
            plt.close()

        #getting original/suggested/final timestamp and original distance
        timestamp_original = lapTimestamp_li
        timestamp_suggested = []
        timestamp_final = []
        distance_original = []

        print('after: ', lapTimestamp_li)
        print('after: ', lapDistance_li)

        for i in range(len(lapRange_df_li)):
            temp = lapRange_df_li[i][((lapRange_df_li[i]['speed'] == 0) & (lapRange_df_li[i]['power'] == 0)) | ((lapRange_df_li[i]['speed'] == 0) & (lapRange_df_li[i]['power'].isnull().all()))]
            try:
                timestamp_suggested.append(temp.iloc[0].name)
            except:
                print(ID+' something is very wrong - suggested')
            try:
                timestamp_final.append(temp.iloc[0].name.date())
            except:
                print(ID+' something is very wrong - final')
            try:
                distance_original.append(lapRange_df_li[i].loc[lapTimestamp_li[i], 'distance'])
            except:
                print(ID+' something is very wrong - distance')
            
        #allVals is the values over in one array.
        allVals = []
        for i in range(len(timestamp_original)):
            try:
                allVals.append([timestamp_original[i], timestamp_suggested[i], timestamp_final[i], distance_original[i]])
            except:
                print(ID+' something is very wrong - allVals')

        #table which is shown. might change the names.
        # removed lap for now, should add back later
        headerTable = ['original timestamp','suggested timestamp','corrected timestamp','original distance']
        #print(tt.to_string(allVals, header=headerTable)) #need help, looks ugly

        #tableDF, used in csv.
        dataTableDF = [headerTable] + allVals
        tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('original timestamp')
        tableDF.to_csv('correlation_{}'.format(ID)+'.csv')

        #basicly lager data/ID folders hvis ikke eksisterer. Hvis data/id/csv fil eksisterer, slett og flytt den nye der. Samme med bilder. Ser sykt ut då hahah.
        if not os.path.exists('../data/{}'.format(ID)):
            os.makedirs('../data/{}'.format(ID))
        if os.path.exists('../data/{}/correlation_{}.csv'.format(ID,ID)):
            os.remove('../data/{}/correlation_{}.csv'.format(ID,ID))
        os.rename('./correlation_{}.csv'.format(ID), '../data/{}/correlation_{}.csv'.format(ID,ID))
        for i in range(len(lapTimestamp_li)):
            if os.path.exists('../data/{}/lap{}pd{}.png'.format(ID,i,ID)):
                os.remove('../data/{}/lap{}pd{}.png'.format(ID,i,ID))
            os.rename('./lap{}pd{}.png'.format(i,ID), '../data/{}/lap{}pd{}.png'.format(ID,i,ID))

#Problemer med blant annet disse deltakerne:
#50305, vil bruke verdiene 2 og 3 for lap 2 og 3. Legge inn default dict for verdi 1.
#50497, ser ut som at han har trykket rett, men er akuratt under 1%
#50810, VELDIG FEIL??


#Problemet for 50305 er måten dette er implementert på. Måten dette programmet kjører på er at den prøver å finne rett verdi for hvert lap etter hverandre. Dette fører til at hvis vi ikke finner rett verdi til lap1, vil programmet potentsielt kaste vekk rett verdi for lap2.

#f.eks:
#ID: 50305
#Before: [32830.63, 67961.41, 75119.39]

    # lagrer forrige verdi:
    #- prevDist = 32830.63

    # fjerner denne verdien:
    #- [67961.41, 75119.39]

    # bytter ut neste verdi med dict:
    #- [33623.22, 75119.39]
    #- prevDist = None

    # Mister derfor verdien 67961, som kunne blitt brukt til lap2.
    #Kan være at det bare er bedre å endre hele framgangsmåten her.


pd_plot_lap_save_corr_new()

In [ ]:
def make_out_folders(ID):
    # Make output folder and error folder
    if not os.path.exists('../data/{}'.format(ID)):
        os.makedirs('../data/{}'.format(ID))
    if not os.path.exists('../data/{}/errors'.format(ID)):
        os.makedirs('../data/{}/errors'.format(ID))

def error_number_of_laps(ID, length):
    with open('../data/{}/errors/incorrectlapcount'.format(ID), 'w') as f:
        f.write(str(length))
        f.close()

def error_2std(ID,i,data,mean,std):
    with open('../data/{}/errors/2std'.format(ID), 'a') as f:
        #values separated by , entries seperated by :
        f.write(str(i)+','+str(data)+','+str(mean)+','+str(std)+':')
        f.close()

def error_autorepair(ID,lap):
    with open('../data/{}/errors/autorepair'.format(ID), 'a') as f:
        #should never exsist without 2std error
        f.write(str(lap)+':')

In [ ]:
# Returns a distance list with the correct number of laps
# from a list that had too many
def find_best_fitting_lap(laps,mean_dict,dist_li):
    ret_li = []
    for i in range(laps):
        closeness = 100000000
        lap = 'a'
        for j in range(len(dist_li)):
            temp = abs(mean_dict[i] - dist_li[j])
            if temp < closeness:
                closeness = temp
                lap = j
        ret_li.append(dist_li[lap])
        if lap != i:
            print('.')
            #error here (used lap bla for bla)
    return ret_li

# Returns a distance list with the correct number of laps
# from a list that had too few
def find_missing_lap(laps,mean_dict,dist_li):
    ret_li = find_best_fitting_lap(laps, mean_dict, dist_li)
    for i in range(len(ret_li)):
        try:
            temp = ret_li[i] - ret_li[i-1]
            if temp == 0:
                ret_li[i] = mean_dict[i]
                #error here, which deletes error from find_best_fitting_lap()
        except:
            continue
    return ret_li

# Use distance list and dataframe to find time list
def get_time_li_from_dist(df,li):
    ret_li = []
    for i in range(len(li)):
        ret_li.append(df.index[df['distance'] == li[i]])
        if type(ret_li[i]) == pd.DatetimeIndex:
            ret_li[i] = pd.Timestamp(ret_li[i][0])
    return ret_li

# Finds the longest list among any number of lists
def longest_list(*args):
    ret = 0
    for argument in args:
        try:
            if len(argument) > ret:
                ret = len(argument)
        except:
            continue
    return ret

# Finds the longest list and adds every value into a single array
# If one or more lists are out of bounds, an empty string is added instead
def lists_to_single(length, *args):
    ret = []
    for i in range(length):
        temp_li = []
        for argument in args:
            try:
                temp = argument[i]
            except:
                temp = ''
            temp_li.append(temp)
        ret.append(temp_li)
    return ret

In [ ]:
# %run ../../lib/statistic-functions.ipynb

mean_dict, std_dict = dist_mean_std()
# print(mean_dict)
# print(std_dict)

### Current problems 

- lap ranges are bad
    - this might be an issue with how they are made
    - have to do extensive testing to fix
    - check_range seems to always be empty, which is wrong
- actual stop detection seems to not work properly, which makes suggested useless
- 50187 and others create lap plot zoomed out for some reason. Probably error from start/stop

In [ ]:
# Debug
global debugs

def pd_plot_lap_save_corr_non_destructive():
    mean_dict, std_dict = dist_mean_std()
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    
    global debugs

    # Change if different race
    correct_number_of_laps = 4

    # Time befor and after a lap (used when plotting a lap)
    timedelta_after = pd.Timedelta("00:00:30")
    timedelta_before = pd.Timedelta("00:00:15")

    for participant in participant_files:    

        # Get ID and create folders
        ID = participant[:-4]  
        make_out_folders(ID)

        # Debug
        print(ID)
        skiplist = []
        if str(ID) in str(skiplist):
            continue

        debugs = []
        debugs.append('Debugs start')
        debugs.append(ID)

        # Get dataframe and remove completly empty line from start of dataframe
        participant_df = add_df[ID]
        participant_df = participant_df[~participant_df.isnull().all(1)]
        
        # Find where lap changes value by 1 and make a list of the timestamps and of the distance.
        lapTimestamp_li = list(participant_df.index[participant_df['lap'].diff() == 1])
        lapDistance_li = list(participant_df['distance'][participant_df['lap'].diff() == 1])

        # Check number of laps, create error file if wrong
        if len(lapTimestamp_li) != correct_number_of_laps:
            error_number_of_laps(ID, len(lapTimestamp_li))
            debugs.append('laplen not ok')
        else:
            corr_dist_li = lapDistance_li
            corr_time_li = lapTimestamp_li

        # If there are too many laps, use the ones that fits best
        if len(lapDistance_li) > correct_number_of_laps:
            corr_dist_li = find_best_fitting_lap(correct_number_of_laps, mean_dict, lapDistance_li)
            try:
                corr_time_li = get_time_li_from_dist(participant_df,corr_dist_li)
            except:
                print(str(ID)+': no timestamps matches distance - fatal error')
                print('This probably means someone did not finish the race')
                continue

        # If there are too few laps, find which one(s) are missing and add those
        if len(lapDistance_li) < correct_number_of_laps:
            corr_dist_li = find_missing_lap(correct_number_of_laps, mean_dict, lapDistance_li)
            try:
                corr_time_li = get_time_li_from_dist(participant_df,corr_dist_li)
            except:
                print(str(ID)+': no timestamps matches distance - fatal error')
                print('This probably means someone did not finish the race')
                continue

        # Now that we have a laplist, we need to see if they are valid BEFORE plotting
        bigbad = [False]
        for i in range(len(corr_time_li)):
            try:
                check_start = participant_df.index.get_loc(corr_time_li[i])
                check_stopp = participant_df.index.get_loc(corr_time_li[i])
                check_range = participant_df.iloc[check_start:check_stopp]
            except:
                print(str(ID)+' !!!FATAL!!! has somehow illegal lap '+str(i))
            try:
                check_start = participant_df.index.get_loc(corr_time_li[i] - timedelta_before)
            except:
                print(str(ID)+' has invalid start at lap '+str(i))
                debugs.append('startlap bad: '+str(i))
            try:
                check_stopp = participant_df.index.get_loc(corr_time_li[i] + timedelta_after)
            except:
                print(str(ID)+' has invalid end at lap '+str(i))
                debugs.append('endlap bad: '+str(i))
            try:
                check_range = participant_df.iloc[check_start:check_stopp]
                f = open('out.txt', 'w')
                for element in check_range:
                    f.write(element.to_string())
                    f.write('\n')
                f.close()
            except:
                print(type(check_start))
                print(check_start)
                print(type(check_stopp))
                print(str(ID)+' has somehow illegal lap range '+str(i))
            try:
                temp = check_range[((check_range['speed'] <= 0.5) & (check_range['power'] <= 0.5)) | ((check_range['speed'] <= 0.5) & (check_range['power'].isnull().all()))]
                if temp == pd.DataFrame():
                    raise ValueError
            except:
                bigbad[0] = True
                #throw error to pdf
                bigbad.append(i)
        break
        # If there is no actual stop at the lap, we need to find one, if possible
        if bigbad[0]:
            print(str(ID)+': has big bad: '+str(bigbad))
            debugs.append('bigbad: '+str(bigbad))

        # List of dataframes, where dataframes contain a range around each lap
        lapRange_df_li = []
        for i in range(correct_number_of_laps):
            try:
                startRow = participant_df.index.get_loc(corr_time_li[i]-timedelta_before)
            except:
                print(ID+'  something is very wrong - startRow')
                try:
                    startRow = participant_df.index.get_loc(corr_time_li[i])
                except:
                    print(ID+': shit hit the fan, sorry')
                    print('i is:'+str(i)+'lapTimstamp_li looks like this:')
                    print(lapTimestamp_li)
                    print(corr_time_li)
            try:
                endRow = participant_df.index.get_loc(corr_time_li[i]+timedelta_after)
            except:
                print(ID+'  something is very wrong - endRow')
                endRow = participant_df.index.get_loc(corr_time_li[i])

            lapRange_df_li.append(participant_df.iloc[startRow:endRow])

        # Create plot for every stop
        for i in range(len(lapRange_df_li)):
            fig, ax = plt.subplots()

            trans = ax.get_xaxis_transform()

            ax2 = ax.twinx()
            ax3 = ax.twinx()
            ax4 = ax.twinx()
            ax5 = ax.twinx()
            ax6 = ax.twinx()

            rightPosition = 1.00
            leftPosition = -0.20

            rightPosition = ax_modification(ax2,'right', rightPosition)
            rightPosition = ax_modification(ax3,'right', rightPosition)
            leftPosition = ax_modification(ax4,'left', leftPosition)
            rightPosition = ax_modification(ax5,'right', rightPosition)
            leftPosition = ax_modification(ax6,'left', leftPosition)
            
            plot_title = "lap {}: -15,+30 pd {}".format(i,ID)

            lapRange_df_li[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title=plot_title)
            lapRange_df_li[i]['speed'].plot(ax=ax2, color='blue',legend=False)
            lapRange_df_li[i]['power'].plot(ax=ax3, color='green',legend=False)
            lapRange_df_li[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
            lapRange_df_li[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
            lapRange_df_li[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)    
            ax.axvline((lapTimestamp_li[i]), color='black', linestyle='--', lw=2)
            ax.text((lapTimestamp_li[i]),.5,(lapTimestamp_li[i]),transform=trans)

            #dårlige runtime, trenge sikkert ikkje sjekka kver einaste value.
            #bør tenka på ein bedre måte
            check_ylim(ax,lapRange_df_li[i],'accumulated_power')
            check_ylim(ax2,lapRange_df_li[i],'speed')
            check_ylim(ax3,lapRange_df_li[i],'power')
            check_ylim(ax4,lapRange_df_li[i],'altitude')
            check_ylim(ax5,lapRange_df_li[i],'distance')

            y_lim_zero(ax)
            y_lim_zero(ax2)
            y_lim_zero(ax3)
            y_lim_zero(ax4)
            y_lim_zero(ax5)
            y_lim_zero(ax6)

            ax_style(ax,'Accumulated Power','red')
            ax_style(ax2,'Speed','blue')
            ax_style(ax3,'Power','green')
            ax_style(ax4,'Altitude','orange')
            ax_style(ax5,'Distance','lightcoral')
            ax_style(ax6,'Heartrate','purple')
            
            #bbox_inches makes it possible to see all y axis in image.
            #all the y axis didnt show pdf when incrementing rightposition and leftposition.
            #however think we need to resize it to fit in the pdf.
            fig.savefig("../data/{}/lap{}pd{}.png".format(ID,i,ID), format="png", bbox_inches='tight')
            #trying to hide plots from notebook
            plt.close()

        #getting original/suggested/final timestamp and original distance
        timestamp_original = lapTimestamp_li
        timestamp_suggested = []
        timestamp_final = []
        distance_original = lapDistance_li

        for i in range(len(lapRange_df_li)):
            temp = lapRange_df_li[i][((lapRange_df_li[i]['speed'] <= 0.5) & (lapRange_df_li[i]['power'] <= 0.5)) | ((lapRange_df_li[i]['speed'] <= 0.5) & (lapRange_df_li[i]['power'].isnull().all()))]
            try:
                timestamp_suggested.append(temp.iloc[0].name)
            except:
                print(ID+' something is very wrong - suggested')
                #should throw pdf error here
                timestamp_suggested.append(timestamp_original[i])
            try:
                timestamp_final.append(temp.iloc[0].name.date())
            except:
                timestamp_final.append(timestamp_original[i].date())

            
        #allVals is the values over in one array.
        allvals_len = longest_list(timestamp_original,timestamp_suggested,timestamp_final,distance_original)
        allVals = lists_to_single(allvals_len,timestamp_original,timestamp_suggested,timestamp_final,distance_original)

        # removed lap for now, should add back later
        headerTable = ['original timestamp','suggested timestamp','corrected timestamp','original distance']

        #tableDF, used in csv.
        dataTableDF = [headerTable] + allVals
        tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('original timestamp')
        tableDF.to_csv('correlation_{}'.format(ID)+'.csv')

        if os.path.exists('../data/{}/correlation_{}.csv'.format(ID,ID)):
            os.remove('../data/{}/correlation_{}.csv'.format(ID,ID))
        os.rename('./correlation_{}.csv'.format(ID), '../data/{}/correlation_{}.csv'.format(ID,ID))



#Problemer med blant annet disse deltakerne:
#50305, vil bruke verdiene 2 og 3 for lap 2 og 3. Legge inn default dict for verdi 1.
#50497, ser ut som at han har trykket rett, men er akuratt under 1%
#50810, VELDIG FEIL??


#Problemet for 50305 er måten dette er implementert på. Måten dette programmet kjører på er at den prøver å finne rett verdi for hvert lap etter hverandre. Dette fører til at hvis vi ikke finner rett verdi til lap1, vil programmet potentsielt kaste vekk rett verdi for lap2.

#f.eks:
#ID: 50305
#Before: [32830.63, 67961.41, 75119.39]

    # lagrer forrige verdi:
    #- prevDist = 32830.63

    # fjerner denne verdien:
    #- [67961.41, 75119.39]

    # bytter ut neste verdi med dict:
    #- [33623.22, 75119.39]
    #- prevDist = None

    # Mister derfor verdien 67961, som kunne blitt brukt til lap2.
    #Kan være at det bare er bedre å endre hele framgangsmåten her.

if True:
    try:
        pd_plot_lap_save_corr_non_destructive()
    except:
        print(debugs)
else:
    pd_plot_lap_save_corr_non_destructive()

In [ ]:
def pd_plot_full():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    for participant in participant_files:
        ID = participant[:-4]  

        if str(ID) != str(50766):
            continue
        #for now, we are only looking at 50338
        add_df_338 = add_df[ID]
        # 15
        #Sjekker der alle verdiene ikke er NaNs. 
        #Gjør det samme som din tidligere funksjon, bare med timestamp som index i betraktning :P.
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]

        #Gjør frekvensen om til rett, slik at det alltid blir timestamp as index
        add_df_338 = add_df_338.asfreq('s')
    
        rightPosition = 1.00
        leftPosition = -0.10

        fig, ax = plt.subplots()

        ax2 = ax.twinx()
        ax3 = ax.twinx()
        ax4 = ax.twinx()
        ax5 = ax.twinx()
        ax6 = ax.twinx()

        ax.autoscale(enable=True, axis='x', tight=True)
        ax2.autoscale(enable=True, axis='x', tight=True)
        ax3.autoscale(enable=True, axis='x', tight=True)
        ax4.autoscale(enable=True, axis='x', tight=True)
        ax5.autoscale(enable=True, axis='x', tight=True)
        ax6.autoscale(enable=True, axis='x', tight=True)


        rightPosition = ax_modification(ax2,'right', rightPosition) - 0.1
        rightPosition = ax_modification(ax3,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax4,'left', leftPosition) + 0.1
        rightPosition = ax_modification(ax5,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax6,'left', leftPosition) + 0.1

        add_df_338['accumulated_power'].plot(ax=ax, color='red',legend=False, title="Full race, pd {}".format(ID), figsize=(10,3))
        add_df_338['speed'].plot(ax=ax2, color='blue',legend=False)
        add_df_338['power'].rolling(window=45).mean().plot(ax=ax3, color='green',legend=False)
        add_df_338['altitude'].plot(ax=ax4, color='orange',legend=False)
        add_df_338['distance'].plot(ax=ax5, color='lightcoral',legend=False)
        add_df_338['heart_rate'].plot(ax=ax6, color='purple',legend=False)

        trans = ax.get_xaxis_transform()


        ax_style(ax,'Accumulated Power','red')
        ax_style(ax2,'Speed','blue')
        ax_style(ax3,'Power','green')
        ax_style(ax4,'Altitude','orange')
        ax_style(ax5,'Distance','lightcoral')
        ax_style(ax6,'Heartrate','purple')

        y_lim_zero(ax)
        y_lim_zero(ax2)
        y_lim_zero(ax3)
        y_lim_zero(ax4)
        y_lim_zero(ax5)
        y_lim_zero(ax6)


        fig.savefig("fullracepd{}.png".format(ID), format="png", bbox_inches='tight')

        if not os.path.exists('../data/{}'.format(ID)):
            os.makedirs('../data/{}'.format(ID))
        if os.path.exists('../data/{}/fullracepd{}.png'.format(ID,ID)):
            os.remove('../data/{}/fullracepd{}.png'.format(ID,ID))
        os.rename('./fullracepd{}.png'.format(ID), '../data/{}/fullracepd{}.png'.format(ID,ID))




pd_plot_full()

In [ ]:

#def pd_plot_lap_save_corr():
#    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
#    ID = '50338'
#    rightPosition = 1.00
#    leftPosition = -0.10

#    fig, ax = plt.subplots()
#    fig.subplots_adjust(right=2)

#    ax2 = ax.twinx()
#    ax3 = ax.twinx()
#    ax4 = ax.twinx()
#    ax5 = ax.twinx()
#    ax6 = ax.twinx()

#    rightPosition = ax_modification(ax2,'right', rightPosition)
#    rightPosition = ax_modification(ax3,'right', rightPosition)
#    leftPosition = ax_modification(ax4,'left', leftPosition)
#    rightPosition = ax_modification(ax5,'right', rightPosition)
#    leftPosition = ax_modification(ax6,'left', leftPosition)

#    lapDates = add_df[ID].index[add_df[ID]['lap'].diff()==1]
    #create new pd lap_df. Contains only +30, -15 sec laps, 1 ekstra for nans.
    #lap_df, gir en ny dataframe. Denne inneholder de tre lap_dfs dataframe.
#    lap_dfs = get_lapdf(add_df, ID, 15, 32, lapDates)
#    lap_df = pd.concat(lap_dfs)
#    lap_df.index = map(lambda x: str(x)[-8:], lap_df.index)

#    lap_df['accumulated_power'].plot(ax=ax, color='red',legend=False, title="all laps -15,+30 pd 50338")
#    lap_df['speed'].plot(ax=ax2, color='blue',legend=False)
#    lap_df['power'].plot(ax=ax3, color='green',legend=False)
#    lap_df['altitude'].plot(ax=ax4, color='orange',legend=False)
#    lap_df['distance'].plot(ax=ax5, color='lightcoral',legend=False)
#    lap_df['heart_rate'].plot(ax=ax6, color='purple',legend=False)

#    ax_style(ax,'Accumulated Power','red')
#    ax_style(ax2,'Speed','blue')
#    ax_style(ax3,'Power','green')
#    ax_style(ax4,'Altitude','orange')
#    ax_style(ax5,'Distance','lightcoral')
#    ax_style(ax6,'Heartrate','purple')

    #getting correlation values, now values and changeable values.
#    nowVals = get_now_val(lapDates, ID, add_df)
#    corrVals = get_corr_val(lap_dfs)
#    changVals = [date.date() for date in lapDates]

    #allVals is the values over in one array.
#    allVals = []
#    for i in range(len(corrVals)):
#        allVals.append([changVals[i]] + [corrVals[i]] + nowVals[i])

    #table which is shown. might change the names.
#    headerTable = ['timestamp correlation changeable','timestamp correlation suggestion','timestamp now','lap','distance']
#    print(tt.to_string(allVals, header=headerTable))

    #tableDF, used in csv.
#    dataTableDF = [headerTable] + allVals
#    tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('timestamp correlation changeable')
#   tableDF.to_csv('correlation_{}'.format(ID)+'.csv')


#pd_plot_lap_save_corr()


In [ ]:
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
def pd_plot_corr():
    for participant in participant_files:
        ID = participant[:-4]
        add_df_338_new = add_df[ID]
        add_df_338_new = add_df_338_new[~add_df_338_new.isnull().all(1)]

        corrDF = pd.read_csv('../data/{}/correlation_{}.csv'.format(ID,ID)).set_index("corrected timestamp")
        corrTS = get_corrTS(corrDF)

        lapRange_df_li = []
        timedelta_30s = pd.Timedelta("00:00:30")
        timedelta_15s = pd.Timedelta("00:00:15")
        for i in range(len(corrTS)):
            startRow = add_df_338_new.index.get_loc(corrTS[i]-timedelta_15s)
            endRow = add_df_338_new.index.get_loc(corrTS[i]+timedelta_30s)
            lapRange_df_li.append(add_df_338_new.iloc[startRow:endRow])

        for i in range(len(lapRange_df_li)):
            rightPosition = 1.00
            leftPosition = -0.20

            fig, ax = plt.subplots()

            ax2 = ax.twinx()
            ax3 = ax.twinx()
            ax4 = ax.twinx()
            ax5 = ax.twinx()
            ax6 = ax.twinx()

            rightPosition = ax_modification(ax2,'right', rightPosition)
            rightPosition = ax_modification(ax3,'right', rightPosition)
            leftPosition = ax_modification(ax4,'left', leftPosition)
            rightPosition = ax_modification(ax5,'right', rightPosition)
            leftPosition = ax_modification(ax6,'left', leftPosition)

            lapRange_df_li[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title="corr lap {} -15,+30 pd {}".format(i, ID))
            lapRange_df_li[i]['speed'].plot(ax=ax2, color='blue',legend=False)
            lapRange_df_li[i]['power'].plot(ax=ax3, color='green',legend=False)
            lapRange_df_li[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
            lapRange_df_li[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
            lapRange_df_li[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)

            trans = ax.get_xaxis_transform()
            ax.axvline((corrTS[i]), color='black', linestyle='--', lw=2)
            ax.text((corrTS[i]),.5,(corrTS[i]),transform=trans)

            check_ylim(ax,lapRange_df_li[i],'accumulated_power')
            check_ylim(ax2,lapRange_df_li[i],'speed')
            check_ylim(ax3,lapRange_df_li[i],'power')
            check_ylim(ax4,lapRange_df_li[i],'altitude')
            check_ylim(ax5,lapRange_df_li[i],'distance')

            ax_style(ax,'Accumulated Power','red')
            ax_style(ax2,'Speed','blue')
            ax_style(ax3,'Power','green')
            ax_style(ax4,'Altitude','orange')
            ax_style(ax5,'Distance','lightcoral')
            ax_style(ax6,'Heartrate','purple')

# pd_plot_corr()